# [CPT12] Competition 1 - Order Brushing

Score: 0.81093

In [1]:
from joblib import Parallel, delayed

In [2]:
import pandas as pd
import datetime as dt

In [3]:
def in_between(my_date, date1, date2):
    return True if date1 <= my_date <= date2 else False

def conc_ratio(df_conc):
    orders = df_conc.orderid.values
    users = df_conc.userid.value_counts()
    
    return len(orders)/len(users.index), users

def user_ratio(shop_users):
    ratio = shop_users/shop_users.sum()
    return ratio.sort_values(ascending=False)

In [4]:
def getPredictions(df_shop):
    # increasing filter size will increase score but also increase run-time
    if len(df_shop) > 80:
        pred = str(0)
        PREDICTIONS.append(pred)
        return pred
    if df_shop.shape[0] < 3:
        pred = str(0)
        PREDICTIONS.append(pred)
        return pred

    df_shop = df_shop.sort_values('event_time')
    
    search_shops = dict()
    for _, row in df_shop.iterrows():
        start_time = row.event_time
        end_time = row.event_time + dt.timedelta(hours=1)

        df_conc = pd.DataFrame()
        df_conc = df_conc.append(row)

        for _, row2 in df_shop.iterrows():
            if row.orderid == row2.orderid:
                continue
            if in_between(row2.event_time, start_time, end_time):
                df_conc = df_conc.append(row2)

        search_shops[(start_time, end_time)] = df_conc

    # transactions tagged as order brushing
    shop_users = pd.Series(dtype='int64')
    for new_df in search_shops.values():
        cr, users = conc_ratio(new_df)
        if cr >= 3:
            shop_users = shop_users.append(users)
    if len(shop_users) == 0:
        pred = str(0)
    else:
        suspicious = user_ratio(shop_users)
        tag_buyer = suspicious.loc[suspicious.values==suspicious.max()].index.astype('int')

        if len(tag_buyer) > 1:
            pred = '&'.join(str(i) for i in sorted(tag_buyer))
        else:
            pred = str(tag_buyer[0])
    return pred

In [5]:
df = pd.read_csv('Copy of Copy of order_brush_order.csv')
df.event_time = df.event_time.astype('datetime64')

In [6]:
result = Parallel(n_jobs=-1, verbose=1)(delayed(getPredictions)(df_shop) for _, df_shop in df.groupby(['shopid']))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 440 tasks      | elapsed:    8.9s
[Parallel(n_jobs=-1)]: Done 1146 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 1816 tasks      | elapsed:   43.9s
[Parallel(n_jobs=-1)]: Done 2716 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 3624 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 4604 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 6088 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 7892 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 10512 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 18770 out of 18770 | elapsed:  4.9min finished


In [17]:
predictions = {}
for (shopid, _), users in zip(df.groupby('shopid'), result):
    predictions[shopid] = users

In [18]:
new_pred_df = (pd.DataFrame
               .from_dict(predictions, orient='index')
               .reset_index()
               .rename({'index':'shopid', 0:'userid'},
                       axis=1)
              )

In [19]:
new_pred_df.to_csv('submission3.csv', index=False)

In [22]:
new_pred_df[new_pred_df['userid']!='0']

,shopid,userid
40,10402,77819
57,10536,672345&672345
111,42472,740844
114,42818,170385453
129,76934,190449497
...,...,...
17401,203531250,114282846
17960,204225676,198662175
18155,208696908,214111334
18557,210197928,52867898
